In [ ]:
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

import WUBRG
import consts
from LineColors import LineColors
from FormatMetadata import SETS, FORMATS, SET_CONFIG
from JSONHandler import JSONHandler
from FormatMetadata import FormatMetadata
from RawDataFetcher import RawDataFetcher

# Objects

In [ ]:
from datetime import date, time, datetime, timedelta
import pandas as pd

import WUBRG
import consts
from JSONHandler import JSONHandler
from RawDataFetcher import RawDataFetcher

class NewRawDataHandler:
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self._FETCHER = RawDataFetcher(SET, FORMAT)
        
        self._GROUPED_ARCHTYPE_HISTORY_FRAME = None
        self._SINGLE_ARCHTYPE_HISTORY_FRAME = None
        self._CARD_HISTORY_FRAME = None
        
        self._GROUPED_ARCHTYPE_SUMMARY_FRAME = None
        self._SINGLE_ARCHTYPE_SUMMARY_FRAME = None
        self._CARD_SUMMARY_FRAME = None
    
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    @property
    def FORMAT(self):
        """The queue type."""
        return self._FORMAT
    
    
    @property
    def GROUPED_ARCHTYPE_HISTORY_FRAME(self):
        """The daily data about how decks, grouped by number of colours, performs."""
        if self._GROUPED_ARCHTYPE_HISTORY_FRAME is None:
            check_for_updates()
        return self._GROUPED_ARCHTYPE_HISTORY_FRAME
    
    @property
    def SINGLE_ARCHTYPE_HISTORY_FRAME(self):
        """The daily data for each deck archetype."""
        if self._SINGLE_ARCHTYPE_HISTORY_FRAME is None:
            check_for_updates()
        return self._SINGLE_ARCHTYPE_HISTORY_FRAME
    
    @property
    def CARD_HISTORY_FRAME(self):
        """The daily data for individual card performance."""
        if self._CARD_HISTORY_FRAME is None:
            check_for_updates()
        return self._CARD_HISTORY_FRAME
    
    
    @property
    def GROUPED_ARCHTYPE_SUMMARY_FRAME(self):
        """The overall data, about how decks, grouped by number of colours, performs."""
        if self._GROUPED_ARCHTYPE_SUMMARY_FRAME is None:
            check_for_updates()
        return self._GROUPED_ARCHTYPE_SUMMARY_FRAME
    
    @property
    def SINGLE_ARCHTYPE_SUMMARY_FRAME(self):
        """The overall data, for each deck archetype."""
        if self._SINGLE_ARCHTYPE_SUMMARY_FRAME is None:
            check_for_updates()
        return self._SINGLE_ARCHTYPE_SUMMARY_FRAME
    
    @property
    def CARD_SUMMARY_FRAME(self):
        """The overall data, about individual card performance."""
        if self._CARD_SUMMARY_FRAME is None:
            check_for_updates()
        return self._CARD_SUMMARY_FRAME
    
    
    def panadafy_card_dict(self, card_dict: dict) -> pd.DataFrame:
        """
        Turns a dictionary into a DataFrame, with some data cleaning applied.
        :param card_dict: The dictionary containing card data for a colour group
        :return: A DataFrame filled with the cleaned card data
        """
        frame = pd.DataFrame.from_dict(card_dict)
        frame = frame.rename(columns=consts.STAT_NAMES)

        # If there's no data, make a blank frame and return it.
        if len(card_dict) == 0:
            return frame

        frame = frame.set_index('Name')
        
        for col in ["GP WR", "OH WR", "GD WR", "GIH WR", "GND WR", "IWD"]:
            frame[col] = frame[col] * 100

        frame = frame.drop(['sideboard_game_count', 'sideboard_win_rate', 'url', 'url_back'], axis=1)
        frame['Rarity'] = frame['Rarity'].map(consts.RARITY_ALIASES)
        frame = frame.round(3)
        return frame
    
    def panadafy_meta_dict(self, meta_dict: dict) -> pd.DataFrame:
        """
        Turns a dictionary into a DataFrame, with some data cleaning applied.
        :param card_dict: The dictionary containing card data for a colour group
        :return: A DataFrame filled with the cleaned card data
        """
        frame = pd.DataFrame.from_dict(meta_dict)
        frame = frame.rename(columns=consts.META_COLS)

        # If there's no data, make a blank frame and return it.
        if len(meta_dict) == 0:
            return frame, frame.copy()
        
        frame['Name'] = frame['Color Name']
        frame = frame.set_index('Name')
        frame['Win %'] = round((frame['Wins'] / frame['Games']) * 100, 2)
        frame['Splash'] = frame['Color Name'].str.contains("Splash", case=False)
        frame['Colors'] = frame['Color Name'].map(lambda x: x.replace(' + Splash', ''))
        frame = frame[['Colors', 'Splash', 'Wins', 'Games', 'Win %', 'is_summary']]
        
        summary_frame = frame[frame['is_summary'] == True].copy()
        summary_frame = summary_frame.drop(['is_summary'], axis=1)
        summary_frame['Colors'] = summary_frame['Colors'].map(WUBRG.COLOR_COUNT_MAP)

        archetype_frame = frame[frame['is_summary'] == False].copy()
        archetype_frame = archetype_frame.drop(['is_summary'], axis=1)
        archetype_frame['Colors'] = archetype_frame['Colors'].map(lambda x: x[0: (x.find('(') if x.find('(') != -1 else len(x))].strip())
        archetype_frame['Colors'] = archetype_frame['Colors'].map(lambda x: x.replace('Mono-', ''))
        archetype_frame['Colors'] = archetype_frame['Colors'].map(WUBRG.COLOR_ALIASES)
        archetype_frame['Name'] = archetype_frame['Colors']
        archetype_frame = archetype_frame.set_index('Name')

        return summary_frame, archetype_frame

        
    def gen_hist(self):
        """Populates and updates the three 'HISTORY' properties."""
        hist_meta, hist_card = self._FETCHER.get_set_data()
        
        grouped_arch_frame_dict = dict()
        single_arch_frame_dict = dict()
        card_frame_dict = dict()
        
        for date in hist_meta:
            grouped_arch_frame_dict[date], single_arch_frame_dict[date] = self.panadafy_meta_dict(hist_meta[date])
        grouped_arch_frame = pd.concat(grouped_arch_frame_dict, names=["Date", "Name"])
        single_arch_frame = pd.concat(single_arch_frame_dict, names=["Date", "Name"])
        
        for date in hist_card:
            color_dict = dict()
            for color in hist_card[date]:
                color_dict[color] = self.panadafy_card_dict(hist_card[date][color])
            card_frame_dict[date] = pd.concat(color_dict, names=["Deck Colors", "Name"])
        card_frame = pd.concat(card_frame_dict, names=["Date", "Deck Colors", "Name"])
            
        self._GROUPED_ARCHTYPE_HISTORY_FRAME = grouped_arch_frame
        self._SINGLE_ARCHTYPE_HISTORY_FRAME = single_arch_frame
        self._CARD_HISTORY_FRAME = card_frame

    def gen_summary(self):
        """Populates and updates the three 'SUMMARY' properties."""
        hist_meta, hist_card = self._FETCHER.get_summary_data()
               
        grouped_arch_frame, single_arch_frame = self.panadafy_meta_dict(hist_meta)

        color_dict = dict()
        for color in hist_card:
            color_dict[color] = self.panadafy_card_dict(hist_card[color])
        card_frame = pd.concat(color_dict, names=["Deck Colors", "Name"])
            
        self._GROUPED_ARCHTYPE_SUMMARY_FRAME = grouped_arch_frame
        self._SINGLE_ARCHTYPE_SUMMARY_FRAME = single_arch_frame
        self._CARD_SUMMARY_FRAME = card_frame
    
    def check_for_updates(self):
        """Populates and updates all data properties."""
        self.gen_hist()
        self.gen_summary()

In [ ]:
class NewRawDataWrapper:   
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self._DATA = NewRawDataHandler(SET, FORMAT)
        self._DATA.check_for_updates()
    
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._FORMAT
    
    @property
    def DATA(self):
        """The object which comtains the data about the set and format."""
        return self._DATA
    
    
    def deck_group_frame(self, date=slice(None, None, None), name=slice(None, None, None), summary=False):
        """Returns a subset of the 'GROUPED_ARCHTYPE' data as a DataFrame."""
        if summary:
            return self.DATA.GROUPED_ARCHTYPE_SUMMARY_FRAME.loc(axis=0)[pd.IndexSlice[name]]
        else:
            return self.DATA.GROUPED_ARCHTYPE_HISTORY_FRAME.loc(axis=0)[pd.IndexSlice[date, name]]
     
    def deck_archetype_frame(self, date=slice(None, None, None), name=slice(None, None, None), summary=False):
        """Returns a subset of the 'SINGLE_ARCHTYPE' data as a DataFrame."""
        if summary:
            return self.DATA.SINGLE_ARCHTYPE_SUMMARY_FRAME.loc(axis=0)[pd.IndexSlice[name]]
        else:
            return self.DATA.SINGLE_ARCHTYPE_HISTORY_FRAME.loc(axis=0)[pd.IndexSlice[date, name]]
    
    def card_frame(self, date=slice(None, None, None), deck_colors=slice(None, None, None), name=slice(None, None, None), card_color=None, card_rarity=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        if type(deck_colors) is str:
            deck_colors = WUBRG.get_color_identity(deck_colors)
        
        if summary:
            ret = self.DATA.CARD_SUMMARY_FRAME.loc(axis=0)[pd.IndexSlice[deck_colors, name]]
        else:
            ret = self.DATA.CARD_HISTORY_FRAME.loc(axis=0)[pd.IndexSlice[date, deck_colors, name]]
            
        if card_color:
            card_color = WUBRG.get_color_identity(card_color)
            ret = ret[ret['Color'] == card_color]
            
        if card_rarity:
            ret = ret[ret['Rarity'] == card_rarity]
        
        return ret

In [ ]:
from datetime import date, time, datetime, timedelta
import pandas as pd

import WUBRG
import consts
from JSONHandler import JSONHandler

class RawDataHandler:    
    
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self.FORMAT_METADATA = FormatMetadata(SET, FORMAT)
        
        self._SUMMARY_DICT = dict()
        self._ARCHTYPE_DICT = dict()
        self._CARD_DICTS = dict()
        
        self.SUMMARY_FRAME = None
        self.ARCHTYPE_FRAME = None
        self.CARD_FRAME = None
    
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    
    @property
    def FORMAT(self):
        """The queue type."""
        return self._FORMAT
    
    
    def panadafy_card_dict(self, card_dict: dict) -> pd.DataFrame:
        """
        Turns a dictionary into a DataFrame, with some data cleaning applied.
        :param card_dict: The dictionary containing card data for a colour group
        :return: A DataFrame filled with the cleaned card data
        """
        frame = pd.DataFrame.from_dict(card_dict)
        frame = frame.rename(columns=consts.STAT_NAMES)

        # If there's no data, make a blank frame and return it.
        if len(card_dict) == 0:
            return frame

        frame = frame.set_index('Name')
        
        for col in ["GP WR", "OH WR", "GD WR", "GIH WR", "GND WR", "IWD"]:
            frame[col] = frame[col] * 100

        frame = frame.drop(['sideboard_game_count', 'sideboard_win_rate', 'url', 'url_back'], axis=1)
        frame['Rarity'] = frame['Rarity'].map(consts.RARITY_ALIASES)
        frame = frame.round(3)
        return frame
    
    def panadafy_meta_dict(self, meta_dict: dict) -> pd.DataFrame:
        """
        Turns a dictionary into a DataFrame, with some data cleaning applied.
        :param card_dict: The dictionary containing card data for a colour group
        :return: A DataFrame filled with the cleaned card data
        """
        frame = pd.DataFrame.from_dict(meta_dict)
        frame = frame.rename(columns=consts.META_COLS)

        # If there's no data, make a blank frame and return it.
        if len(meta_dict) == 0:
            return frame, frame.copy()
        
        frame['Name'] = frame['Color Name']
        frame = frame.set_index('Name')
        frame['Win %'] = round((frame['Wins'] / frame['Games']) * 100, 2)
        frame['Splash'] = frame['Color Name'].str.contains("Splash", case=False)
        frame['Colors'] = frame['Color Name'].map(lambda x: x.replace(' + Splash', ''))
        frame = frame[['Colors', 'Splash', 'Wins', 'Games', 'Win %', 'is_summary']]
        
        summary_frame = frame[frame['is_summary'] == True].copy()
        summary_frame = summary_frame.drop(['is_summary'], axis=1)
        summary_frame['Colors'] = summary_frame['Colors'].map(WUBRG.COLOR_COUNT_MAP)

        archetype_frame = frame[frame['is_summary'] == False].copy()
        archetype_frame = archetype_frame.drop(['is_summary'], axis=1)
        archetype_frame['Colors'] = archetype_frame['Colors'].map(lambda x: x[0: (x.find('(') if x.find('(') != -1 else len(x))].strip())
        archetype_frame['Colors'] = archetype_frame['Colors'].map(lambda x: x.replace('Mono-', ''))
        archetype_frame['Colors'] = archetype_frame['Colors'].map(WUBRG.COLOR_ALIASES)
        archetype_frame['Name'] = archetype_frame['Colors']
        archetype_frame = archetype_frame.set_index('Name')

        return summary_frame, archetype_frame
    
    def gen_frames(self):
        self.SUMMARY_FRAME = pd.concat(self._SUMMARY_DICT, names=["Date", "Name"])
        self.ARCHTYPE_FRAME = pd.concat(self._ARCHTYPE_DICT, names=["Date", "Name"])
        temp_dict = dict()
        for date in self._CARD_DICTS:
            temp_dict[date] = pd.concat(self._CARD_DICTS[date], names=["Deck Colors", "Name"])
        self.CARD_FRAME = pd.concat(temp_dict, names=["Date", "Deck Colors", "Name"])
        
    def get_day_data(self, check_date):
        loader = JSONHandler(self.SET, self.FORMAT, check_date)
        str_date = str(check_date)
        print(f'Getting data for {self.SET} {self.FORMAT}, date: {str_date}')
        card_dict, meta_dict = loader.get_day_data()
        
        self._SUMMARY_DICT[str_date], self._ARCHTYPE_DICT[str_date] = self.panadafy_meta_dict(meta_dict)
        temp_dict = dict()
        for color in card_dict:
            temp_dict[color] = self.panadafy_card_dict(card_dict[color])
        self._CARD_DICTS[str_date] = temp_dict
            
        return self._SUMMARY_DICT[str_date], self._ARCHTYPE_DICT[str_date], self._CARD_DICTS[str_date]

    def get_set_data(self):
        check_date = self.FORMAT_METADATA.START_DATE

        run = True        
        while(run):
            if self.FORMAT_METADATA.is_active(check_date):
                self.get_day_data(check_date)
            check_date += timedelta(days=1)
            #TODO: Make this more in line with the time check for general data.
            run = check_date < date.today() 
    
        self.gen_frames()
        return self._SUMMARY_DICT, self._ARCHTYPE_DICT, self._CARD_DICTS
    
    def get_summary_data(self):
        loader = JSONHandler(self.SET, self.FORMAT, None)
        print(f'Getting overall data for {self.SET} {self.FORMAT}')
        card_dict, meta_dict = loader.get_day_data(overwrite=True)
        
        summary, archetype = self.panadafy_meta_dict(meta_dict)
        cards = dict()
        for color in card_dict:
            cards[color] = self.panadafy_card_dict(card_dict[color])
            
        return summary, archetype, cards

In [ ]:
class RawDataWrapper:   
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self._DATA = RawDataHandler(SET, FORMAT)
    
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._FORMAT
    
    
    @property
    def DATA(self):
        """The data about the set and format."""
        return self._DATA
    

    def get_set_data(self):
        self.DATA.get_set_data()
        
    def get_summary_data(self):
        return self.DATA.get_summary_data()
    
    def summary_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.SUMMARY_FRAME.loc(axis=0)[pd.IndexSlice[date, name]]
     
    def archetype_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.ARCHTYPE_FRAME.loc(axis=0)[pd.IndexSlice[date, name]]
    
    def card_frame(self, date=slice(None, None, None), colors=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.CARD_FRAME.loc(axis=0)[pd.IndexSlice[date, colors, name]]

In [ ]:
class DataManager:
    def __init__(self):        
        self.DATA = {s: {f: RawDataWrapper(s, f) for f in FORMATS} for s in SETS}
        
        self._SET = SETS[0]
        self._FORMAT = FORMATS[0]
        self._ACTIVE_DATA = self.DATA[self._SET][self._FORMAT]
        
    @property
    def SET(self):
        """The draft set."""
        return self._SET

    @SET.setter
    def SET(self, value):
        if value in SETS:
            self._SET = value
            self._ACTIVE_DATA = self.DATA[self._SET][self._FORMAT]
        else:
            raise ValueError(f"Inavlid set: '{value}'")
        
    @property
    def FORMAT(self):
        """The draft format."""
        return self._FORMAT

    @FORMAT.setter
    def FORMAT(self, value):
        if value in FORMATS:
            self._FORMAT = value
            self._ACTIVE_DATA = self.DATA[self.SET][self.FORMAT]
        else:
            raise ValueError(f"Inavlid format: '{value}'")
            
    
    @property
    def ACTIVE_DATA(self):
        """The data for the given set and format."""
        return self._ACTIVE_DATA
    
    def get_set_data(self):
        self.ACTIVE_DATA.get_set_data()
        
    def get_summary_data(self):
        return self.ACTIVE_DATA.get_summary_data()
    
    def summary_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.ACTIVE_DATA.summary_frame(date, name)
     
    def archetype_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.ACTIVE_DATA.archetype_frame(date, name)
    
    def card_frame(self, date=slice(None, None, None), colors=slice(None, None, None), name=slice(None, None, None)):
        if type(colors) is str:
            colors = WUBRG.get_color_identity(colors)
        return self.ACTIVE_DATA.card_frame(date, colors, name)

# Functions

## Card Summary Functions

In [ ]:
def get_card_summary(card_name, colors='', roll=1):
    frame = manager.card_frame(name=card_name, colors=colors)[['GIH WR', 'ALSA', '# GP', 'IWD']]
    frame.index = [tup[0][5:] for tup in frame.index]
    rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
    return rolling

In [ ]:
def plot_card_summary(card_name, colors='', roll=1):
    rolling = get_card_summary(card_name, colors, roll)
    title = f"{card_name}"
    
    col_filt = f"Color Filter: {colors}"
    rol_filt = f"Rolling Average: {roll} Days"
    
    if colors and roll > 1:
        title += f"\n{col_filt}  -  {rol_filt}"
    elif colors:
            title += f"\n{col_filt}"
    elif roll > 1:
            title += f"\n{rol_filt}"
    
    rolling.plot(subplots=True, layout=(2,2), figsize=(12,8), title=title)

## Pick Order Functions

In [ ]:
def graph_pick_stats(card_name, roll=1):
    taken_data = manager.card_frame(name=card_name, colors='')[['ALSA', 'ATA']]
    taken_data.index = [tup[0][5:] for tup in taken_data.index]
    rolling = taken_data.rolling(window=roll, min_periods=1, center=True).mean()
    
    mx = min(max(taken_data.max()) + 0.25, 15) 
    mn = max(min(taken_data.min()) - 0.25, 1)
    tit_str = f"{manager.SET} - {manager.FORMAT}\n{card_name}"
    
    rolling.plot(ylim=(mx, mn), grid=True, title=tit_str)
    
    return taken_data

In [ ]:
def compare_card_evaluations(start_date, end_date):
    def inner_func(date):
        df = manager.card_frame(date=date, colors='')
        df.index = [tup[2] for tup in df.index]
        return df

    first = inner_func(date='2022-02-10')
    last = inner_func(date='2022-02-15')
    diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
    diff['Δ ALSA'] = first['ALSA'] - last['ALSA']
    diff['Δ ATA'] = first['ATA'] - last['ATA']
    return diff[['ALSA', 'Δ ALSA', 'ATA', 'Δ ATA', 'Color', 'Rarity']]

## Archetype Functions

In [ ]:
def get_archetype_frame(colors, roll=1):
    win_rate_frame = manager.archetype_frame(name=colors)
    win_rate_frame.index = [tup[0] for tup in win_rate_frame.index]
    #win_rate_frame = win_rate_frame[['Splash', 'Games', 'Win %']]
    win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False][['Wins', 'Games']]
    rolling = win_rate_frame.rolling(window=roll, min_periods=1, center=True).mean().round()
    rolling['Win %'] = round((rolling['Wins'] / rolling['Games']) * 100, 2)
    rolling['Avg. Win%'] = [get_winrate(idx) for idx in win_rate_frame.index]
    rolling['Win % Offset'] = rolling['Win %'] - rolling['Avg. Win%']
    return rolling

## Winrate Functions

In [ ]:
def get_winrate(day=None):
    if day:
        return manager.summary_frame().loc[(day, 'All Decks')]['Win %']
    else:
        # TODO: Get the overall winrate 
        return None

In [ ]:
def get_archetype_winrate_history(color_filter=None, roll=1):
    d = dict()
    for col in WUBRG.COLOR_PAIRS:
        d[col] = get_archetype_frame(col)[col_name]
    d['AVG'] = get_archtype_winrates_frame('UR')['Avg. Win%']
        
    test_frame = pd.DataFrame.from_dict(d)
    test_frame.index = [idx[5:] for idx in test_frame.index]
    if color_filter:
        col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col] + ['AVG'] 
        test_frame = test_frame[col_filt]
    
    rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
    return rolling

In [ ]:
def plot_archetype_winrate_history(color_filter='', roll=1):
    test_frame = get_archetype_winrate_history(color_filter, roll)
    lc = LineColors()
    title = "Archetpye Winrates"
    
    col_filt = f"Color Filter: {color_filter}"
    rol_filt = f"Rolling Average: {roll} Days"
    
    if color_filter and roll > 1:
        title += f"\n{col_filt}  -  {rol_filt}"
    elif color_filter:
            title += f"\n{col_filt}"
    elif roll > 1:
            title += f"\n{rol_filt}"
    test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)

## Playrate Functions

In [ ]:
def get_archetype_playrate_history(color_filter=None, roll=1):    
    d = dict()
    for col in WUBRG.COLOR_PAIRS:
        d[col] = get_archetype_frame(col)['Games']

    test_frame = pd.DataFrame.from_dict(d)
    test_frame.index = [idx[5:] for idx in test_frame.index]
    rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
    total = rolling.sum(axis=1)
    playrate = rolling.divide(list(total),axis=0) * 100
    
    if color_filter:
        col_filt = [col for col in WUBRG.COLOR_PAIRS if color_filter in col]
        playrate = playrate[col_filt]
    
    return playrate

In [ ]:
def plot_archetype_playrate_history(color_filter='', roll=1):
    test_frame = get_archetype_playrate_history(color_filter, roll)
    lc = LineColors()
    title = "Archetpye Playrates"
    
    col_filt = f"Color Filter: {color_filter}"
    rol_filt = f"Rolling Average: {roll} Days"
    
    if color_filter and roll > 1:
        title += f"\n{col_filt}  -  {rol_filt}"
    elif color_filter:
            title += f"\n{col_filt}"
    elif roll > 1:
            title += f"\n{rol_filt}"
    test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)

# Initialization

In [ ]:
color_pairs = [''] + [WUBRG.COLOR_ALIASES_SUPPORT['Guilds'][key] for key in WUBRG.COLOR_ALIASES_SUPPORT['Guilds']]
color_pairs

In [ ]:
manager = DataManager()
manager.SET = 'NEO'
manager.FORMAT = 'PremierDraft'

In [ ]:
manager.get_set_data()

In [ ]:
#manager.get_summary_data()

## Current Tests

In [ ]:
fetcher = NewRawDataHandler('NEO', 'PremierDraft')
fetcher.check_for_updates()


fetcher.GROUPED_ARCHTYPE_HISTORY_FRAME
fetcher.SINGLE_ARCHTYPE_HISTORY_FRAME
fetcher.CARD_HISTORY_FRAME

fetcher.GROUPED_ARCHTYPE_SUMMARY_FRAME
fetcher.SINGLE_ARCHTYPE_SUMMARY_FRAME
fetcher.CARD_SUMMARY_FRAME

## Old Tests

In [ ]:
manager.card_frame(name='Virus Beetle', colors='')

In [ ]:
manager.card_frame(name='Leech Gauntlet', colors='')

In [ ]:
day_frame = manager.archetype_frame(date='2022-02-14')
win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False]
day_frame['Games'].sum()
day_frame

In [ ]:
frame = today[2][''].copy()
avg_wr = today[0].loc['All Decks']['Win %']
wu_avg_wr = today[1][today[1]['Colors'] == 'WU'][today[1]['Splash'] == False].iloc[0]['Win %']
#frame['Δ WR'] = None
frame = frame.drop(['# Seen', '# Picked', '# OH', '# GD', '# GIH', '# GND'], axis=1)
frame['(x-μ) GP WR'] = frame['GP WR'] - avg_wr
frame['(x-μ) GIH WR'] = frame['GIH WR'] - avg_wr

frame.sort_values(by=['(x-μ) GP WR'])
frame[frame['(x-μ) GP WR'] < frame['(x-μ) GIH WR']].sort_values(by=['(x-μ) GIH WR'])
frame['Spikiness'] =  frame['GIH WR'] - frame['GP WR']
frame

# Data Graphing and Display

## Card Summary

In [ ]:
plot_card_summary('Eiganjo Exemplar')

In [ ]:
plot_card_summary('Eiganjo Exemplar', 'RW')

In [ ]:
plot_card_summary('Eiganjo Exemplar', 'RW', 3)

In [ ]:
TRGT_CARDS = ['Roaring Earth', 'Tales of Master Seshiro', 'Befriending the Moths']
TRGT_PAIR = ''
ROLL = 3

for CARD in TRGT_CARDS:
    plot_card_summary(CARD, colors=TRGT_PAIR, roll=ROLL)

## Card Pick Order Stats

In [ ]:
graph_pick_stats('Voldaren Epicure', 1)

In [ ]:
graph_pick_stats('Imperial Oath', 1)

In [ ]:
graph_pick_stats('Behold the Unspeakable', 3)

In [ ]:
to_graph = ['Imperial Oath', 'Behold the Unspeakable', 'Virus Beetle', 'Network Disruptor']
for card in to_graph:
    graph_pick_stats(card, 1)

## Archetype Winrate History

In [ ]:
get_archetype_playrate_history(roll=1)

In [ ]:
get_archetype_playrate_history(roll=3)

In [ ]:
plot_archetype_winrate_history('W', 3)
plot_archetype_winrate_history('U', 3)
plot_archetype_winrate_history('B', 3)
plot_archetype_winrate_history('R', 3)
plot_archetype_winrate_history('G', 3)
plot_archetype_winrate_history('', 3)

## Archetype Playrate History

In [ ]:
get_archetype_playrate_history(roll=1)

In [ ]:
get_archetype_playrate_history(roll=3)

In [ ]:
plot_archetype_playrate_history('W', 3)
plot_archetype_playrate_history('U', 3)
plot_archetype_playrate_history('B', 3)
plot_archetype_playrate_history('R', 3)
plot_archetype_playrate_history('G', 3)
plot_archetype_playrate_history('', 3)

## Card Pick Order Changes

In [ ]:
diff = compare_card_evaluations('2022-02-10', '2022-02-16')
commons = diff[diff['Rarity'] == 'C']
uncommons = diff[diff['Rarity'] == 'U']

In [ ]:
SRT_TRG = 'Δ ATA'
commons.sort_values(SRT_TRG, ascending=False).head(20)

In [ ]:
commons.sort_values(SRT_TRG, ascending=True).head(20)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=False).head(10)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=True).head(10)